## 참고: https://hleecaster.com/ml-linear-regression-example/

## 라이브러리 설치, 호출

!pip3 install -U scikit-learn<br>
!pip3 install pandas<br>
!pip3 install numpy<br>
!pip3 install matplotlib<br>
!pip3 install statsmodels<br>

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau

## 데이터 다운로드 (특별할인 판매)

In [2]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/jmnote/zdata/master/logistic-regression/special-sales.csv')
print(df.head())
print(df.shape)

         date weekday  busy_day  high_temperature  special_sales
0  2002-08-05     Mon         0                28              1
1  2002-08-06     Tue         0                24              0
2  2002-08-07     Wed         1                26              0
3  2002-08-08     Thu         0                24              0
4  2002-08-09     Fri         0                23              0
(21, 5)


## Input, Feature 설정

In [3]:
Label = df['special_sales']
InputFeature = df[['busy_day','high_temperature']]

## Keras Logit 모델 fitting

In [4]:
model = Sequential()
model.add(Dense(3, activation='linear', input_shape=(2,)))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=Adam(learning_rate=0.01), metrics=['accuracy'])

## Keras 모델 살펴보기

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 3)                 9         
                                                                 
 dense_1 (Dense)             (None, 1)                 4         
                                                                 
Total params: 13
Trainable params: 13
Non-trainable params: 0
_________________________________________________________________


## Call-back 함수
## 모델 학습

In [6]:
# Call-back 함수
# CheckPoint: Epoch 마다 validation 성능을 검증하여, best performance 일 경우 저장
CP = ModelCheckpoint(filepath='-{epoch:03d}-{loss:.4f}-{accuracy:.4f}.hdf5',
            monitor='loss', verbose=1, save_best_only=True, mode='min')

# Learning Rate 줄여나가기
LR = ReduceLROnPlateau(monitor='loss',factor=0.8,patience=3, verbose=1, min_lr=1e-8)

CALLBACK = [CP, LR]

In [9]:
model.fit(x=InputFeature, y=Label, epochs=100, shuffle=True, batch_size=3, callbacks=CALLBACK)

Epoch 1/100
1/7 [===>..........................] - ETA: 5s - loss: 1.6365 - accuracy: 0.6667
Epoch 1: loss improved from inf to 0.95236, saving model to -001-0.9524-0.6667.hdf5
7/7 [==============================] - 1s 9ms/step - loss: 0.9524 - accuracy: 0.6667 - lr: 0.0100
Epoch 2/100
1/7 [===>..........................] - ETA: 0s - loss: 0.6397 - accuracy: 0.6667
Epoch 2: loss improved from 0.95236 to 0.95050, saving model to -002-0.9505-0.3810.hdf5
7/7 [==============================] - 0s 10ms/step - loss: 0.9505 - accuracy: 0.3810 - lr: 0.0100
Epoch 3/100
1/7 [===>..........................] - ETA: 0s - loss: 0.7592 - accuracy: 0.0000e+00
Epoch 3: loss improved from 0.95050 to 0.67726, saving model to -003-0.6773-0.6190.hdf5
7/7 [==============================] - 0s 10ms/step - loss: 0.6773 - accuracy: 0.6190 - lr: 0.0100
Epoch 4/100
1/7 [===>..........................] - ETA: 0s - loss: 0.8093 - accuracy: 0.6667
Epoch 4: loss did not improve from 0.67726
7/7 [====================

# Model Load 하기 전, hdf5 파일 이름 꼭 확인하기

In [10]:
model.load_weights("-098-0.5575-0.7619.hdf5")


## FLASK 셋팅하기

In [11]:
from flask import Flask
from flask import render_template
from flask import request

In [12]:
app = Flask(__name__)

# FLASK API 구현부분

In [13]:
@app.route('/')
@app.route('/SpecialSale')
def PredictionSpecialSale():
    busy_day = request.args.get('busy_day')
    high_temperature = request.args.get('high_temperature')
    
    print(busy_day, high_temperature)
    
    if busy_day == None or high_temperature == None:
        return render_template('SpecialSale.html', Output = '')
    
    Input = pd.DataFrame({
        'busy_day':[int(busy_day)],
        'high_temperature':[float(high_temperature)]
    })
    ModelOutput = model.predict(Input)[0][0]

    return render_template('SpecialSale.html', Output = ModelOutput)

# Flask, port 5000으로 실행

In [14]:
app.run(host='0.0.0.0', port=5000)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://192.168.0.5:5000/ (Press CTRL+C to quit)
192.168.0.5 - - [07/Jul/2022 10:17:53] "GET / HTTP/1.1" 200 -


None None
1 38
1/1 [==============================] - 0s 241ms/step


192.168.0.5 - - [07/Jul/2022 10:18:01] "GET /SpecialSale?busy_day=1&high_temperature=38 HTTP/1.1" 200 -


In [ ]:
# http://127.0.0.1:5000/SpecialSale/1/38